<a href="https://colab.research.google.com/github/Achmad96/Achmad96/blob/main/fraud_detection_on_transactions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade tensorflow==2.19.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.0/645.0 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 101.2 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.20.0
    Uninstalling tensorboard-2.20.0:
      Successfully uninstalled tensorboard-2.20.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.20.0
    Uninstalling tensorflow-2.20.0:
      Successfully uninstalled tensorflow-2.20.0


In [ ]:
!pip install -U kaleido

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.4 MB/s eta 0:00:00


In [ ]:
!pip install torch torch-geometric scikit-learn matplotlib seaborn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.4 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
print(tf.__version__)

2.19.0


# Load Dataset

In [50]:
from google.colab import drive
from pathlib import Path

drive_path = Path('/content/drive')
drive.mount(str(drive_path))
drive_path /= 'MyDrive'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [51]:
import warnings
warnings.filterwarnings('ignore')

In [52]:
SEED = 42

In [53]:
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix, roc_auc_score, roc_curve, precision_recall_curve
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import shuffle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, Model, models, callbacks, regularizers
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, LSTM, Input

from scipy.spatial.distance import pdist, squareform
import networkx as nx

from imblearn.over_sampling import SMOTE

In [54]:
dataset_root = drive_path / 'Fraud Dataset'

activities_df = pd.read_csv(dataset_root  / "Customer Profiles" / "account_activity.csv")
customers_df = pd.read_csv(dataset_root   / "Customer Profiles" / "customer_data.csv")
fraud_indicators_df = pd.read_csv(dataset_root  / "Fraudulent Patterns" / "fraud_indicators.csv")
sus_activities_df = pd.read_csv(dataset_root    / "Fraudulent Patterns" / "suspicious_activity.csv")
merchants_df = pd.read_csv(dataset_root               / "Merchant Informations" / "merchant_data.csv")
transaction_categories_df = pd.read_csv(dataset_root  / "Merchant Informations" / "transaction_category_labels.csv")
transaction_amounts_df = pd.read_csv(dataset_root  / "Transaction Amounts" / "amount_data.csv")
anomaly_scores_df = pd.read_csv(dataset_root       / "Transaction Amounts" / "anomaly_scores.csv")
transaction_metadata_df = pd.read_csv(dataset_root / "Transaction Data" / "transaction_metadata.csv")
transaction_records_df = pd.read_csv(dataset_root   / "Transaction Data" / "transaction_records.csv")

In [55]:
df = pd.merge(activities_df, customers_df, on='CustomerID')
df = pd.merge(df, transaction_records_df, on='CustomerID')
df = pd.merge(df, sus_activities_df, on='CustomerID')
df = pd.merge(df, transaction_metadata_df, on='TransactionID')
df = pd.merge(df, anomaly_scores_df, on='TransactionID')
df = pd.merge(df, transaction_amounts_df, on='TransactionID')
df = pd.merge(df, transaction_categories_df, on='TransactionID')
df = pd.merge(df, fraud_indicators_df, on='TransactionID')
df = pd.merge(df, merchants_df, on='MerchantID')
df.head()

,CustomerID,AccountBalance,LastLogin,Name,Age,Address,TransactionID,Amount,SuspiciousFlag,Timestamp,MerchantID,AnomalyScore,TransactionAmount,Category,FraudIndicator,MerchantName,Location
0,1102,1628.195364,2024-01-01 00:00:00+07:00,Customer 1102,35,Address 1000,1,14.069680,0,2025-01-01 00:00:00+07:00,2001,0.626593,17.491962,Online,0,Merchant 2001,Location 1000
1,1102,1628.195364,2024-01-01 00:00:00+07:00,Customer 1102,35,Address 1000,1,14.069680,0,2025-01-01 00:00:00+07:00,2001,0.626593,17.491962,Online,0,Merchant 2001,Location 1000
2,1102,1628.195364,2024-01-01 00:00:00+07:00,Customer 1102,35,Address 1000,580,56.141966,0,2025-07-30 23:12:25.945945948+07:00,2580,0.839802,54.688328,Retail,0,Merchant 2580,Location 1579
3,1102,1628.195364,2024-01-01 00:00:00+07:00,Customer 1102,35,Address 1000,580,56.141966,0,2025-07-30 23:12:25.945945948+07:00,2580,0.839802,54.688328,Retail,0,Merchant 2580,Location 1579
4,1102,1628.195364,2024-01-01 00:00:00+07:00,Customer 1102,47,Address 1579,1,14.069680,0,2025-01-01 00:00:00+07:00,2001,0.626593,17.491962,Online,0,Merchant 2001,Location 1000


# Exploratory Data Analysis

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19120 entries, 0 to 19119
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CustomerID         19120 non-null  int64  
 1   AccountBalance     19120 non-null  float64
 2   LastLogin          19120 non-null  object 
 3   Name               19120 non-null  object 
 4   Age                19120 non-null  int64  
 5   Address            19120 non-null  object 
 6   TransactionID      19120 non-null  int64  
 7   Amount             19120 non-null  float64
 8   SuspiciousFlag     19120 non-null  int64  
 9   Timestamp          19120 non-null  object 
 10  MerchantID         19120 non-null  int64  
 11  AnomalyScore       19120 non-null  float64
 12  TransactionAmount  19120 non-null  float64
 13  Category           19120 non-null  object 
 14  FraudIndicator     19120 non-null  int64  
 15  MerchantName       19120 non-null  object 
 16  Location           191

In [ ]:
df.describe().round(3)

,CustomerID,AccountBalance,Age,TransactionID,Amount,SuspiciousFlag,MerchantID,AnomalyScore,TransactionAmount,FraudIndicator
count,19120.000,19120.000,19120.000,19120.000,19120.000,19120.000,19120.000,19120.000,19120.000,19120.000
mean,1487.300,5644.603,40.040,477.834,55.628,0.021,2477.834,0.475,55.295,0.061
std,311.727,2546.418,13.143,284.357,26.172,0.143,284.357,0.290,26.211,0.238
min,1000.000,1045.709,18.000,1.000,10.417,0.000,2001.000,0.000,5.771,0.000
25%,1189.000,3486.150,29.000,237.000,30.899,0.000,2237.000,0.242,30.356,0.000
50%,1496.000,5801.306,39.000,462.000,56.703,0.000,2462.000,0.449,56.467,0.000
75%,1775.000,7964.207,52.000,725.000,77.285,0.000,2725.000,0.737,77.009,0.000
max,1998.000,9997.671,64.000,1000.000,99.975,1.000,3000.000,0.999,103.177,1.000


In [ ]:
df.isnull().sum()

,0
CustomerID,0
AccountBalance,0
LastLogin,0
Name,0
Age,0
Address,0
TransactionID,0
Amount,0
SuspiciousFlag,0
Timestamp,0


In [ ]:
suspicious_flags = df['SuspiciousFlag'].value_counts()
suspicious_flags_map = {
    0: 'Non-Suspicious',
    1: 'Suspicious'
}
suspicious_flags.index = suspicious_flags.index.map(suspicious_flags_map)
fig = px.bar(
    title='Class Distribution for Suspicious Flag',
    x=suspicious_flags.index,
    y=suspicious_flags.to_numpy(),
    labels={'x': 'Suspicious Flag', 'y': 'Count'},
    hover_name=suspicious_flags.index,
    color=suspicious_flags.index,
    color_discrete_sequence=px.colors.qualitative.T10)
fig.show()

In [ ]:
fraud_indicators = df['FraudIndicator'].value_counts()
fraud_indicators_map = {
    0: 'Legit',
    1: 'Fraud'
}
fraud_indicators.index = fraud_indicators.index.map(fraud_indicators_map)
fig = px.bar(
    title='Class Distribution for Fraud Indicator',
    x=fraud_indicators.index,
    y=fraud_indicators.to_numpy(),
    labels={'x': 'Fraud Indicator', 'y': 'Count'},
    hover_name=fraud_indicators.index,
    color=fraud_indicators.index,
    color_discrete_sequence=px.colors.qualitative.T10)
fig.show()

In [ ]:
category_frauds = df[df['FraudIndicator'] == 1]['Category'].value_counts()
fig = px.bar(
    title='Fraud by Category',
    x=category_frauds.index,
    y=category_frauds.to_numpy(),
    labels={'x': 'Fraud Indicator', 'y': 'Count'},
    hover_name=category_frauds.index,
    color=category_frauds.index,
    color_discrete_sequence=px.colors.qualitative.T10)
fig.show()

# Feature Engineering

In [ ]:
df.columns

Index(['CustomerID', 'AccountBalance', 'LastLogin', 'Name', 'Age', 'Address',
       'TransactionID', 'Amount', 'SuspiciousFlag', 'Timestamp', 'MerchantID',
       'AnomalyScore', 'TransactionAmount', 'Category', 'FraudIndicator',
       'MerchantName', 'Location'],
      dtype='object')

In [ ]:
numerical_features = df.select_dtypes(include=['int64', 'float64'])
categorical_features = df.select_dtypes(include=['object'])
print(f"""
Numerical Features:
{numerical_features.columns}

Categorical Features:
{categorical_features.columns}
""")


Numerical Features:
Index(['CustomerID', 'AccountBalance', 'Age', 'TransactionID', 'Amount',
       'SuspiciousFlag', 'MerchantID', 'AnomalyScore', 'TransactionAmount',
       'FraudIndicator'],
      dtype='object')

Categorical Features:
Index(['LastLogin', 'Name', 'Address', 'Timestamp', 'Category', 'MerchantName',
       'Location'],
      dtype='object')



In [ ]:
drop_columns=['TransactionID','MerchantID','MerchantName','Location','CustomerID','Name', 'Age', 'Address']
df = df.drop(drop_columns, axis=1)

In [ ]:
df.head()

,AccountBalance,LastLogin,Amount,SuspiciousFlag,Timestamp,AnomalyScore,TransactionAmount,Category,FraudIndicator
0,1628.195364,2024-01-01 00:00:00+07:00,14.069680,0,2025-01-01 00:00:00+07:00,0.626593,17.491962,Online,0
1,1628.195364,2024-01-01 00:00:00+07:00,14.069680,0,2025-01-01 00:00:00+07:00,0.626593,17.491962,Online,0
2,1628.195364,2024-01-01 00:00:00+07:00,56.141966,0,2025-07-30 23:12:25.945945948+07:00,0.839802,54.688328,Retail,0
3,1628.195364,2024-01-01 00:00:00+07:00,56.141966,0,2025-07-30 23:12:25.945945948+07:00,0.839802,54.688328,Retail,0
4,1628.195364,2024-01-01 00:00:00+07:00,14.069680,0,2025-01-01 00:00:00+07:00,0.626593,17.491962,Online,0


In [ ]:
df['Datetime'] = pd.to_datetime(df['Timestamp'], format='ISO8601')

In [ ]:
df['Hour'] = df['Datetime'].dt.hour
df['LastLogin'] = pd.to_datetime(df['LastLogin'], format='ISO8601')
df['Gap'] = (df['Datetime'] - df['LastLogin']).dt.days.abs()
df.head()

,AccountBalance,LastLogin,Amount,SuspiciousFlag,Timestamp,AnomalyScore,TransactionAmount,Category,FraudIndicator,Datetime,Hour,Gap
0,1628.195364,2024-01-01 00:00:00+07:00,14.069680,0,2025-01-01 00:00:00+07:00,0.626593,17.491962,Online,0,2025-01-01 00:00:00+07:00,0,366
1,1628.195364,2024-01-01 00:00:00+07:00,14.069680,0,2025-01-01 00:00:00+07:00,0.626593,17.491962,Online,0,2025-01-01 00:00:00+07:00,0,366
2,1628.195364,2024-01-01 00:00:00+07:00,56.141966,0,2025-07-30 23:12:25.945945948+07:00,0.839802,54.688328,Retail,0,2025-07-30 23:12:25.945945948+07:00,23,576
3,1628.195364,2024-01-01 00:00:00+07:00,56.141966,0,2025-07-30 23:12:25.945945948+07:00,0.839802,54.688328,Retail,0,2025-07-30 23:12:25.945945948+07:00,23,576
4,1628.195364,2024-01-01 00:00:00+07:00,14.069680,0,2025-01-01 00:00:00+07:00,0.626593,17.491962,Online,0,2025-01-01 00:00:00+07:00,0,366


In [ ]:
X = df.drop(['FraudIndicator','Timestamp','Datetime','LastLogin'],axis=1)
y = df['FraudIndicator']

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
X['Category'] = label_encoder.fit_transform(X['Category'])
X.head()

,AccountBalance,Amount,SuspiciousFlag,AnomalyScore,TransactionAmount,Category,Hour,Gap
0,1628.195364,14.069680,0,0.626593,17.491962,1,0,366
1,1628.195364,14.069680,0,0.626593,17.491962,1,0,366
2,1628.195364,56.141966,0,0.839802,54.688328,3,23,576
3,1628.195364,56.141966,0,0.839802,54.688328,3,23,576
4,1628.195364,14.069680,0,0.626593,17.491962,1,0,366


In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19120 entries, 0 to 19119
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   AccountBalance     19120 non-null  float64
 1   Amount             19120 non-null  float64
 2   SuspiciousFlag     19120 non-null  int64  
 3   AnomalyScore       19120 non-null  float64
 4   TransactionAmount  19120 non-null  float64
 5   Category           19120 non-null  int64  
 6   Hour               19120 non-null  int32  
 7   Gap                19120 non-null  int64  
dtypes: float64(4), int32(1), int64(3)
memory usage: 1.1 MB


In [ ]:
smote = SMOTE(random_state=SEED, k_neighbors=20)
X_resampled, y_resampled = smote.fit_resample(X,y)
X_resampled, y_resampled = shuffle(X_resampled, y_resampled, random_state=SEED)
df_resampled = pd.concat([X_resampled, y_resampled], axis=1)
print(f"""
X-original: {X.shape}, X-resampled: {X_resampled.shape}
y-original: {y.shape}, y-resampled: {y_resampled.shape}""")


X-original: (19120, 8), X-resampled: (35926, 8)
y-original: (19120,), y-resampled: (35926,)


In [ ]:
fraud_indicators = y_resampled.value_counts()
fraud_indicators.index = fraud_indicators.index.map(fraud_indicators_map)
fig = px.bar(
    x=fraud_indicators.index,
    y=fraud_indicators.to_numpy(),
    labels={'x': 'Fraud Indicator', 'y': 'Count'},
    hover_name=fraud_indicators.index,
    color=fraud_indicators.index,
    title='Class Distribution after Oversampling',
    color_discrete_sequence=px.colors.qualitative.T10)
fig.show()

In [ ]:
df_resampled = pd.concat([X_resampled, y_resampled], axis=1)
df_resampled.head()

,AccountBalance,Amount,SuspiciousFlag,AnomalyScore,TransactionAmount,Category,Hour,Gap,FraudIndicator
2980,5747.463501,90.708888,0,0.395131,90.952235,1,4,308,0
30616,2275.337070,95.491621,0,0.945752,98.290320,3,19,161,1
21501,1831.112551,84.599503,0,0.996158,84.617048,4,0,43,1
19535,7964.206737,75.931489,0,0.786206,73.902127,3,21,146,1
34829,4794.406440,88.980684,0,0.826599,87.105345,2,17,97,1


In [ ]:
category_frauds = df_resampled[df_resampled['FraudIndicator'] == 1]['Category'].value_counts()
fig = px.bar(
    title='Fraud by Category',
    x=category_frauds.index,
    y=category_frauds.to_numpy(),
    labels={'x': 'Fraud Indicator', 'y': 'Count'},
    hover_name=category_frauds.index,
    color=category_frauds.index,
    color_discrete_sequence=px.colors.qualitative.T10)
fig.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=SEED)
print(f"Training set: {X_train.shape}")
print(f"Test set: {X_test.shape}")
print(f"Class distribution - Train: {np.bincount(y_train)}")
print(f"Class distribution - Test: {np.bincount(y_test)}")

Training set: (13384, 8)
Test set: (5736, 8)
Class distribution - Train: [12574   810]
Class distribution - Test: [5389  347]


In [ ]:
X_legit = X[y == 0]
X_fraud = X[y == 1]
print(X_legit.shape, X_fraud.shape)

(17963, 8) (1157, 8)


In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
num_nodes = X_train_scaled.shape[0]
input_dim = X_train_scaled.shape[1]
input_dim

8

In [ ]:
X_train_cnn = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
X_test_cnn = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

# Modelling

In [ ]:
model = Sequential()
model.add(Input(shape=(input_dim, 1)))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [63]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 6, 64)          │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 3, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 192)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        12,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 38,021 (148.52 KB)

 Trainable params: 12,673 (49.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 25,348 (99.02 KB)

In [ ]:
model.fit(X_train_cnn, y_train, epochs=10, batch_size=32, verbose=1)

Epoch 1/10
419/419 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9201 - loss: 0.2255
Epoch 2/10
419/419 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9544 - loss: 0.1151
Epoch 3/10
419/419 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9549 - loss: 0.1096
Epoch 4/10
419/419 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9611 - loss: 0.0977
Epoch 5/10
419/419 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9699 - loss: 0.0843
Epoch 6/10
419/419 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9680 - loss: 0.0864
Epoch 7/10
419/419 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9722 - loss: 0.0821
Epoch 8/10
419/419 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9724 - loss: 0.0835
Epoch 9/10
419/419 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9759 - loss: 0.0733
Epoch 10/10
419/419 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9761 - loss: 0.0698


In [64]:
y_pred = (model.predict(X_test_cnn) > 0.5).astype('int32')

print("CNN Classification Report:")
print(classification_report(y_test, y_pred))

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
CNN Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.99      0.99      5389
           1       0.81      0.74      0.78       347

    accuracy                           0.97      5736
   macro avg       0.90      0.87      0.88      5736
weighted avg       0.97      0.97      0.97      5736



In [66]:
cm = confusion_matrix(y_test, y_pred)
fig = px.imshow(
    cm,
    text_auto=True,
    labels=dict(x="Predicted", y="True", color="Count"),
    x=['Non-Fraud', 'Fraud'],
    y=['Non-Fraud', 'Fraud'],
    color_continuous_scale='Blues'
)
fig.show()

In [67]:
accuracy_score(y_test, y_pred)

0.974023709902371

# INFERENCES

In [62]:
test_data = {
    'AccountBalance': [10000.0, 450.0, 15000.0, 1200.0, 3500.0, 1200.0, 500.0, 45000.0, 3000.0, 8000.0, 200.0, 6000.0, 100.0, 7000.0, 50.0],
    'Amount': [55.0, 400.0, 2500.0, 150.0, 25.0, 950.0, 480.0, 5000.0, 850.0, 70.0, 300.0, 120.0, 600.0, 30.0, 1000.0],
    'SuspiciousFlag': [0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1],
    'AnomalyScore': [0.08, 0.99, 0.15, 0.92, 0.12, 0.98, 0.99, 0.12, 0.95, 0.1, 0.85, 0.2, 0.97, 0.05, 0.99],
    'TransactionAmount': [55.0, 400.0, 2500.0, 150.0, 25.0, 950.0, 480.0, 5000.0, 850.0, 70.0, 300.0, 120.0, 600.0, 30.0, 1000.0],
    'Category': ['Food', 'Online', 'Travel', 'Online', 'Retail', 'Online', 'Travel', 'Retail', 'Online', 'Food', 'Online', 'Food', 'Online', 'Travel', 'Online'],
    'Hour': [13, 3, 15, 2, 10, 3, 2, 14, 1, 18, 5, 20, 4, 11, 23],
    'Gap': [2, 850, 10, 500, 5, 5, 10, 2, 12, 1, 300, 8, 200, 3, 700]
}

# The ground truth for evaluation (0 = Legitimate, 1 = Fraud)
ground_truth = [0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1]

# Create a DataFrame from the inference data
inference_df = pd.DataFrame(test_data)

inference_df['Category'] = label_encoder.transform(inference_df['Category'])
inference_features = inference_df[['AccountBalance', 'Amount', 'SuspiciousFlag', 'AnomalyScore', 'TransactionAmount', 'Category', 'Hour', 'Gap']]
inference_features_scaled = scaler.transform(inference_features)

# Reshape the inference features to match the CNN model's expected input shape (samples, features, 1)
inference_features_cnn = inference_features_scaled.reshape((inference_features_scaled.shape[0], inference_features_scaled.shape[1], 1))

predictions = model.predict(inference_features_cnn)
predictions = (predictions > 0.5).astype('int32')
print(confusion_matrix(ground_truth, predictions))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
[[4 3]
 [0 8]]


In [68]:
accuracy_score(ground_truth, predictions)

0.8